In [1]:
from collections import defaultdict
from datetime import datetime
import torch
import numpy as np
import pandas as pd
from datetime import datetime

In [14]:
ml_1m_path = '../../../datasets/ml-1m'
PATH_TO_PROCESSED_DATA = 'processed/'
dataAfter = 'preprocessed_data/'
dayTime = 86400

In [4]:
file_path = ml_1m_path + '/ratings.dat'
data_ratings = pd.read_csv(file_path, sep='::', engine='python',
                   names=['SessionID', 'ItemID', 'Rating', 'Time'], index_col='SessionID').drop('Rating', axis=1, inplace=False)

## SASRec handling
Only use implicit feedback in the sequence of items

In [5]:
data_ratings

,ItemID,Time
SessionID,,
1,1193,978300760
1,661,978302109
1,914,978301968
1,3408,978300275
1,2355,978824291
...,...,...
6040,1091,956716541
6040,1094,956704887
6040,562,956704746


In [6]:
def removeShortSessions(data):
    #delete sessions of length < 1
    sessionLen = data.groupby('SessionID').size() #group by sessionID and get size of each session
    data = data[np.in1d(data.SessionID, sessionLen[sessionLen > 1].index)]
    return data

In [7]:
actions = data_ratings
actions = actions.groupby('SessionID').filter(lambda x: len(x)>=2)
actions = actions.groupby('ItemID').filter(lambda  x: len(x)>= 5)
# item length > 4
actions = actions.groupby('SessionID').filter(lambda x: len(x)>=2)
# session_lengths > 1
train = actions[(actions.index % 5 != 0)]
test = actions[(actions.index % 5 == 0)]

In [8]:
train

,ItemID,Time
SessionID,,
1,1193,978300760
1,661,978302109
1,914,978301968
1,3408,978300275
1,2355,978824291
...,...,...
6039,1081,956705989
6039,1083,956706051
6039,1086,956706182


In [9]:
test

,ItemID,Time
SessionID,,
5,2987,978243170
5,2333,978242607
5,1175,978244759
5,39,978245037
5,288,978246585
...,...,...
6040,1091,956716541
6040,1094,956704887
6040,562,956704746


In [16]:
print('Testing Set has', len(test), 'Events, ', test.index.nunique(), 'Sessions, and', test.ItemID.nunique(), 'Items\n\n')
test.to_csv(dataAfter + 'ml-1m-Test.txt', sep=',', index=True)

Testing Set has 198466 Events,  1208 Sessions, and 3370 Items




In [11]:
timeMax = train.Time.max()
sessionMaxTime = train.groupby('SessionID').Time.max()
# temporal split
sessionTrain = sessionMaxTime[sessionMaxTime < (timeMax - dayTime)].index #training split is all sessions that ended before the last 2nd day
sessionValid = sessionMaxTime[sessionMaxTime >= (timeMax - dayTime)].index #validation split is all sessions that ended during the last 2nd day
trainTR = train[np.in1d(train.index, sessionTrain)]
trainVD = train[np.in1d(train.index, sessionValid)]
trainVD = trainVD[np.in1d(trainVD.ItemID, trainTR.ItemID)]

In [12]:
sessionMaxTime

SessionID
1       978824351
2       978300174
3       978298504
4       978294282
6       978239019
          ...    
6034    956712388
6036    956755196
6037    956801840
6038    956717204
6039    956758029
Name: Time, Length: 4832, dtype: int64

In [56]:
trainTR

,ItemID,Time
SessionID,,
1,1193,978300760
1,661,978302109
1,914,978301968
1,3408,978300275
1,2355,978824291
...,...,...
6039,1081,956705989
6039,1083,956706051
6039,1086,956706182


In [59]:
trainVD = trainVD.groupby('SessionID').filter(lambda x: len(x)>=2)

In [60]:
trainVD

,ItemID,Time
SessionID,,
403,647,976308336
403,648,976292055
403,3863,983048243
403,2052,976306625
403,2126,976303458
...,...,...
5948,1088,959956945
5948,1096,957190795
5948,1097,957291769


In [18]:
print('Training Set has', len(trainTR), 'Events, ', trainTR.index.nunique(), 'Sessions, and', trainTR.ItemID.nunique(), 'Items\n\n')
trainTR.to_csv(dataAfter + 'ml-1m-Train.txt', sep=',', index=True)
print('Validation Set has', len(trainVD), 'Events, ', trainVD.index.nunique(), 'Sessions, and', trainVD.ItemID.nunique(), 'Items\n\n')
trainVD.to_csv(dataAfter + 'ml-1m-Valid.txt', sep=',', index=True)

Training Set has 798405 Events,  4826 Sessions, and 3416 Items


Validation Set has 2740 Events,  6 Sessions, and 1411 Items


